In [1]:
%matplotlib qt

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

from scipy.stats import skewnorm
import arviz as az

In [3]:
rng = np.random.default_rng(10)
x = skewnorm(3, loc=5, scale=2).rvs(size=50, random_state=rng)
y = - np.ones_like(x)

## Computing histogram animation

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from functools import partial

fig, ax = plt.subplots()
bins = az.stats.density_utils.get_bins(x)
patches = [p for p in ax.hist(x, bins=bins, histtype="bar")[2]]
rugs = [ax.plot([], [], "|k", markersize=10)[0] for _ in x]
bin_edges, = ax.plot([], [], "^b", markersize=5)

ax.legend(
    handles=[patches[0], bin_edges, rugs[0]],
    labels=["Histogram", "Bin edges", "Data"]
)

def init():
    ax.set_title('Histogram')
    ax.spines.right.set_color('none')
    ax.spines.bottom.set_position('zero')
    ax.spines.top.set_color('none')
    ax.xaxis.set_ticks_position('bottom')
    ax.set_ylim(-1.8, 12.5);
    for p in patches:
            p.set_height(0)
    bin_edges.set_data([], [])
    for r in rugs:
        r.set_data([], [])
    return tuple([bin_edges, *patches, *rugs])

def update(frame, samples):
    step, idx = frame
    if step == "rug":
        for rug, sample in zip(rugs, samples):
            rug.set_data([sample], [-1])
            rug.set_color("black")
    elif step == "edges":
        bin_edges.set_data(bins, np.zeros_like(bins))
    elif step == "counting":
        rugs[idx].set(color="red", markersize=30)
        if idx > 0:
            rugs[idx-1].set(color="lightgray", markersize=10)
        bin = int((samples[idx] - bins[0]) // (bins[1] - bins[0]))
        patches[bin].set_height(patches[bin].get_height() + 1)
    elif step == "cleanup":
        rugs[-1].set(color="lightgray", markersize=10)
    return tuple([bin_edges, *patches, *rugs])

frames = [
    *[("wait", None)]*2,
    ("rug", None),
    *[("wait", None)]*4,
    ("edges", None),
    *[("wait", None)]*4,
    *[("counting", i) for i in range(len(x))],
    ("cleanup", None),
]

ani = FuncAnimation(
    fig, partial(update, samples=x),
    frames=frames,
    init_func=init,
    blit=True,
    interval=300,
    repeat_delay=2000,
)

ani.save("histogram.mp4", dpi=300)

plt.show()

## Histogram info diagram